# AdVantage
adVantage is a collection of tools to be able to detect moving aircraft in a video from the vantage platform. Below is a walkthrough how to perform the tracking process.

https://vantage.earthi.world/explore/

### Download yolov5 repository

Yolov5 repository needs to be installed before the AdVantage example will work. You will need to clone the repository into the current working directory.

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

### Authenticate with the Vantage platform
To interact with the api you will need an API key that can be generated on the vantage platform. If you do not have this please generate a key from your account area.
https://vantage.earthi.world/advanced/#/account . Your username will be the email address you used to sign up with

In [ ]:
import getpass
from vantage_api.services import VantageServiceApi

user = input('Vantage Email Address: ')
apiKey = getpass.getpass('API Key:')

vantageApi = VantageServiceApi(user, apiKey)

### Searching for an Airport
You are able to search for airports using the airport database. If you know the IATA code you can use the code below to find the airport. The result will give to longitude and latiute information you can use to search for videos on the Vantage platform. A couple of codes that can be used here:

* AKL

In [ ]:
from airport_database.airports import Airports
from vantage_api.utils import GeoRequest

iataCode = input('IATA Code: ')
airports = Airports()
airport = airports.findByIATACode(iataCode.upper())
print('Airport: '+airport.name)
print('Latitude: '+str(airport.lat_decimal))
print('Longitude: '+str(airport.lon_decimal))

geoData = GeoRequest()
geoData.fromBoundingBox(airport.boundingBox(halfSideInKm=10))

features = vantageApi.getSearch(geoData).getFeatures()
for key in features.keys():
    print(key)

chosenFeature = None
while chosenFeature == None:
    chosenKey = input('Enter a Key to Process: ')    
    if chosenKey in features.keys():
        chosenFeature = features[chosenKey]        

print('Chosen: '+chosenFeature)

### Download Video From Vantage Platform
Downloading the image from the vantage platform requires you to create a job and wait for its result. The API wrapper on AdVantage will allow you to do this in one command.

You can also download a stablised video using the `stabliseVideo` service.

Please choose one of two pieces of code to start the download. 

**Please note it can take quite some time to download these videos from the Vantage platform**

In [ ]:
outputFiles = vantageApi.downloadFiles(chosenFeature,'./input', verbose=True)
print(outputFiles)

In [ ]:
outputFiles = vantageApi.stabliseVideo(chosenFeature,'./input', verbose=True)
print(outputFiles)

### Detect Aircraft Movements
The AdVantage platform is written to run via a pipeline. This allows you to easily create tasks for each frame of the video and perform tasks

In [2]:
import os.path
import zipfile
import sys
sys.path.insert(0, './yolov5')
from advantage.app import AdVantage
from advantage.handlers import *

#Test file if downloads on Vantage are not working - File must be in the input folder
#chosenFeature = 'https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003b9.zip'

app = AdVantage
cwd = '.'
outputDir = os.path.join(cwd, 'output')

if not os.path.exists(outputDir):
    os.makedirs(outputDir)
   
filename, file_extension = os.path.splitext(chosenFeature)
filename = os.path.basename(filename)
isZip = file_extension == '.zip'
if isZip:
    print('unzipping files')
    with zipfile.ZipFile(os.path.join(cwd,'input',filename+'.zip'), 'r') as zip_ref:
        zip_ref.extractall('input/')


pipelines = [
    #Verbose(),
    VideoWriter(outputDir+'/'+filename+'.mp4'),
    YoloProcessor(os.path.join(cwd,'input','best.pt')), 
    VideoPredictionVisulisation(),
]

if isZip:
    pipelines.append(VideoAttachGeoData(os.path.join(cwd,'input',filename+'_geometry.xml')))

outputVideoPath = os.path.join(cwd,'input', filename+'.mp4')
print('Processing Video Please Wait')
result = app.process_video(outputVideoPath, app.pipeline_factory(pipelines))
print('Video Created: '+outputVideoPath)   




unzipping files


YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.2+cu102 CPU

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs


Processing Video Please Wait


# Pipeline todo
- get video from vantage platform (or wget a pre-downloaded video for POC)
- split into frames  or load video frame per frame using open cv
- run yolo on each frame recording bbox of each image in yolo format
- convert yolo format bbox to geographic format using frame metadata and projection
    - display on map for POC
- create binary image which is geo based
- run particle filter or something similar using CV2 to track objects
- use tracked objects positioning etc. to calculate speed and heading
- display this both in a map and on the original video